# 베이즈 텍스트 분석
- 메일이 스팸메일인지 아닌지 예측

In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics

데이터 읽기
--

- data from 'https://www.kaggle.com/ishansoni/sms-spam-collection-dataset/data'
- download the data file (spam.csv) from the above site and save it in the current directory
- if you use Colab, just upload the file to the colab working directory

In [5]:
sms = pd.read_csv('spam.csv', encoding = 'latin-1')   
sms.head()

v1                                                 v2 Unnamed: 2  \
0   ham  Go until jurong point, crazy.. Available only ...        NaN   
1   ham                      Ok lar... Joking wif u oni...        NaN   
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...        NaN   
3   ham  U dun say so early hor... U c already then say...        NaN   
4   ham  Nah I don't think he goes to usf, he lives aro...        NaN   

  Unnamed: 3 Unnamed: 4  
0        NaN        NaN  
1        NaN        NaN  
2        NaN        NaN  
3        NaN        NaN  
4        NaN        NaN

In [6]:
# Drop the extra columns and rename columns to make it look the same as the text

sms = sms.drop(labels = ["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis = 1)
sms.columns = ["type", "text"]

In [7]:
sms.head(3)

type                                               text
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...

In [8]:
sms.shape

(5572, 2)

In [9]:
sms['type'].value_counts()

ham     4825
spam     747
Name: type, dtype: int64

In [10]:
sms_corpus = sms['text'] 

In [11]:
type(sms_corpus)

pandas.core.series.Series

In [12]:
sms['type'].value_counts().plot(kind='pie', explode=[0, 0.1], autopct='%1.1f%%')
plt.legend(['ham', 'spam'])
plt.show()

## 단어-문서 행렬 (Word-Document Matrix) 생성

In [13]:
vectorizer = CountVectorizer(min_df=10)  # 10번 이하로 발생한 단어 무시
X = vectorizer.fit_transform(sms_corpus)

In [14]:
X.shape

(5572, 1016)

In [16]:
vectorizer.get_feature_names_out()[-10:]   # 마지막 단어 10개만

array(['yo', 'you', 'your', 'yours', 'yourself', 'yr', 'yup', 'ì_', 'ìï',
       'û_'], dtype=object)

In [17]:
y = sms['type'] 
y[:5]

0     ham
1     ham
2    spam
3     ham
4     ham
Name: type, dtype: object

In [18]:
# X.toarray(): transform a sparse matrix into a matrix format
X[:5].toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [25]:
# for exercise
maxidx = X[:5].toarray().argmax(axis=1); maxidx
maxcounts = X[:5].toarray().max(axis=1)
maxwords = [vectorizer.get_feature_names()[i] for i in maxidx]
list(zip(maxidx, maxcounts, maxwords ))

[(92, 1, 'available'),
 (460, 1, 'lar'),
 (862, 3, 'to'),
 (728, 2, 'say'),
 (391, 2, 'he')]

In [30]:
# 전체 max 단어
print('max counts: {}'.format(X.toarray().sum(0).max()))
print('index for max counts: {}'.format(X.toarray().sum(0).argmax()))
print('word for max counts: {}'.format(vectorizer.get_feature_names()[X.toarray().sum(0).argmax()]))

max counts: 2242
index for max counts: 862
word for max counts: to


In [31]:
# 전체 5개 max 단어
iidx = X.toarray().sum(0).argsort()[-5:][::-1]
print(iidx)
print([vectorizer.get_feature_names()[i] for i in iidx])
print([X.toarray().sum(0)[i] for i in iidx])

[ 862 1007  836   67  432]
['to', 'you', 'the', 'and', 'in']
[2242, 2240, 1328, 979, 898]


## Bayes Algorithm 적용

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X.toarray(), y, test_size=0.25)

In [33]:
clf = GaussianNB() 
clf.fit(X_train, y_train)
print(clf.score(X_test, y_test).round(4))

0.7997


In [34]:
y_predict = clf.predict(X_test) 
print(metrics.classification_report(y_test, y_predict)) 

              precision    recall  f1-score   support

         ham       0.99      0.78      0.87      1191
        spam       0.42      0.93      0.57       202

    accuracy                           0.80      1393
   macro avg       0.70      0.85      0.72      1393
weighted avg       0.90      0.80      0.83      1393



In [39]:
print(metrics.confusion_matrix(y_test, y_predict))   
# known to be in i, predicted in j
# (0,0):true neg, (1,0): false neg, (1,1): true pos, (0,1): false pos)
#             0(no)     1(spam)
#   0(no)      942          255
#   1(spam)     8           188        recall_spam = 188/196

[[926 265]
 [ 14 188]]


In [38]:
print(metrics.classification_report(y_test, y_predict))

              precision    recall  f1-score   support

         ham       0.99      0.78      0.87      1191
        spam       0.42      0.93      0.57       202

    accuracy                           0.80      1393
   macro avg       0.70      0.85      0.72      1393
weighted avg       0.90      0.80      0.83      1393



## 다른 알고리즘 적용

In [41]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
mod1 = SGDClassifier()
mod1.fit(X_train, y_train)
mod1.score(X_test, y_test)

0.9806173725771715

In [42]:
mod1 = RandomForestClassifier(n_estimators=100)
mod1.fit(X_train, y_train)
mod1.score(X_test, y_test)

0.9806173725771715

In [43]:
mod1 = DecisionTreeClassifier()
mod1.fit(X_train, y_train)
mod1.score(X_test, y_test)

0.9691313711414213

In [44]:
mod1 = LogisticRegression()
mod1.fit(X_train, y_train)
mod1.score(X_test, y_test)

0.9791816223977028

# Exercise

- CountVectorizer
  - 정규표현식을 사용해 토큰을 추출한다.
  - 모두 소문자로 변환시키기 때문에 good, Good, gOod이 모두 같은 특성이 된다.
  - 의미 없는 특성을 많이 생성하기 때문에 적어도 두 개의 문서에 나타난 토큰만을 사용한다.
  - min_df로 토큰이 나타날 최소 문서 개수를 지정할 수 있다.

In [57]:
# countvectorizer example
from sklearn.feature_extraction.text import CountVectorizer
corpus = [
        'This is the first document.',
        'This document is the second document.',
        'And this is the third one.',
        'Is this the first document?',
]
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)

In [58]:
# print(X)
print(vectorizer.get_feature_names_out())     # same as vectorizer.get_feature_names()
print(X.toarray())
X.shape

['and' 'document' 'first' 'is' 'one' 'second' 'the' 'third' 'this']
[[0 1 1 1 0 0 1 0 1]
 [0 2 0 1 0 1 1 0 1]
 [1 0 0 1 1 0 1 1 1]
 [0 1 1 1 0 0 1 0 1]]


(4, 9)

In [55]:
# list 연습
list = [1,2,3,4,5]
good=[0,2]
good_list = [list[i] for i in good]
good_list

[1, 3]

In [54]:
# argmax 연습
a = np.arange(6).reshape(2,3) + 10
print(a)
np.argmax(a), np.argmax(a, axis=0), np.argmax(a, axis=1)


[[10 11 12]
 [13 14 15]]


(5, array([1, 1, 1]), array([2, 2]))